# Cars Sold in Germany From 2018 to 2021 (Analytic Study)

## Introduction

The Car Market is Very Intersting To Analyise Because of the variation in Customers Needs and how each car brand 
try to fill the needs , So When We look in car sales Report We Can determine Which cars Sold the Most and Why... 
When We Answer This Questions That Will Helps The Cars Manufacturers To Focus On Producing Cars That Suitable For The 
Customers!

### The Data Set Used For This Study 

Germany Cars Dataset From Kaggle ->
Dataset scraped from AutoScout24 with information about new and used cars In Germany.


The Link For DataSet : https://www.kaggle.com/datasets/ander289386/cars-germany

### About The DataSet

The Data is Structured as Table in CSV File 

There are **9 features** describing **46405 vehicles** for sale in Germany, registration year being from **2011 to 2021**

**The Features are :** make, model , mileage , fuel , horse power , price , offertype , gear , year 

### Tools Used For The Study

- Google Spread Sheet : For Showing The Data And Some Basic Editing
- SQL Server 
- SQL Server Managment System : Apply Queries To Clean and Get Insights
- Power BI : To Visual The Insights And Building The Dashboard

### What This Study Should Answer

When Studying The Data Provided I came Up With This Questions That Should Be Answered :

1. What Are The TOP 10 Car Brands Base On There Sales ?
2. How Each Car Brand From The Previous Question Makes Up For Overall Sales And What is The Average Price For the Cars ?
3. What Are The Top Models Sold For Each Brand ?
4. Which OfferType Make The Most For Cars Sold ?
5. Which Gear Make The Most For Cars Sold ?
6. What is The Link Between The OfferType and Gear For The Brands and Models ?

**This Questions Should Give Insights About The Overall Data Which Will Give Space To Answer More Questions In Progress**

## Basic Tweaking For The Data

![rowsingooglespread](imgwsps.png)

At First Glance We Can Show The Features But We Have Small Problem (The Rows Not Have ID'S For Indexing 
Which We Can Solve it When Loading To The Data Base Table)

## Data Storing And Cleaning 

### **1. Store The Data In The Database**

As We Know The CSV File Not Have ID Column So When Importing The Data Will Imported To Table Without The Column

**To Solve This We Can Use This Query To add The Column req_id to The Table:**

    ALTER TABLE car_sales ADD req_id INT IDENTITY(1,1) PRIMARY KEY;


![reqidwrongpos](/img/reqidwrongpos.png)

** Note That The req_id not in the right Position and it is Not The Primary Key

For Solving This Problem We Can Create New Table add Constraints , Define Primary Key And Order The Columns In Meaningful Way

The Queries For Creating Inserting Drop And Change Name:


    CREATE TABLE car_sales_new (
        req_id INT IDENTITY(1,1) PRIMARY KEY,
        make VARCHAR(50) NULL,
        model VARCHAR(50) NULL,
        year SMALLINT NULL,
        mileage INT NULL,
        fuel VARCHAR(50) NULL,
        gear VARCHAR(50) NULL,
        hp SMALLINT NULL,
        offertype VARCHAR(50) NULL,
        price INT NULL
    );


    INSERT INTO car_sales_new (req_id, make, model, year, mileage, fuel, gear, hp, offertype, price)
    SELECT req_id, make, model, year, mileage, fuel, gear, hp, offertype, price FROM car_sales;



    DROP TABLE car_sales;



    EXEC sp_rename 'car_sales_new', 'car_sales';


![columninright](img/columninright.png)

Now We Can Study The Data

Count Null Values 

    SELECT COUNT(*) AS Null_Row_Count
    FROM car_sales
    WHERE mileage IS NULL 
       OR make IS NULL 
       OR model IS NULL 
       OR fuel IS NULL 
       OR gear IS NULL 
       OR offertype IS NULL 
       OR price IS NULL 
       OR hp IS NULL 
       OR year IS NULL;


**Number Of Rows Contains Null Values : 334** Which Present 0.719 % From All Data

**Note : The Study Focus On 3 Years Only So if We Limit By Year Using 


SELECT Count (*)  
    FROM car_sales  
    WHERE CAST(year AS SMALLINT) >= (SELECT MAX(CAST(year AS SMALLINT)) FROM car_sales) - 1;

**The Size Of Data Sample is : 8467**

**The Rows Contain Null Values : 62** Which Present 0.73% From all Data

and We Found That The Majority Of The Rows That Contains Null Are in The Model And Gear

When Using This Query :

    SELECT make, COUNT(*)  AS nulls
    FROM car_sales  
    WHERE CAST(year AS SMALLINT) >= (SELECT MAX(CAST(year AS SMALLINT)) FROM car_sales) - 1 
      AND (
        mileage IS NULL 
        OR make IS NULL 
        OR model IS NULL 
        OR fuel IS NULL 
        OR gear IS NULL 
        OR offertype IS NULL 
        OR price IS NULL 
        OR hp IS NULL 
        OR year IS NULL
      )Group by make ORDER BY nulls DESC;

We Found That The Best Approach Is To Delete Rows Have Null Values it Will Not Effect The Study

#### 2. Create A View To Process The Data ( Not Effecting The Original Data ) 

    CREATE OR ALTER VIEW car_post_process AS  
    SELECT  
        req_id,  
        mileage,  
        make,  
        model,  
        fuel,  
        gear,  
        offertype,  
        price,  
        hp,  
        CAST(year AS SMALLINT) AS sale_year  
    FROM car_sales Where CAST(year AS SMALLINT) >= (SELECT MAX(CAST(year AS SMALLINT)) FROM car_sales) - 3 ;


Create The View With 3 Years Span

**Drop The Rows Contains Null**

### Top 10 car sold Brand Base On The Cars_Sold and Total Revenue


    SELECT TOP(10)
        make, COUNT(*) AS cars_sold ,
        SUM(car_price) AS total_sales,
        ROUND( (SUM(car_price) * 100.0) / (SELECT SUM(car_price) FROM car_post_process), 2) AS 
        percentage_of_total_revenue  
        FROM car_post_process 
        GROUP BY make  
        ORDER BY total_sales DESC;

![top10brands](img/top10brands.png)

### What Are The Top Models Sold For Each Brand ?



    SELECT 
        make,  
        model,  
        COUNT(*) AS cars_sold  
    FROM car_post_process  
    WHERE make IN ('Volkswagen', 'Audi', 'Ford', 'Mercedes-Benz', 'Skoda', 'Renault', 'Hyundai', 'BMW', 'Opel', 'SEAT')  
    GROUP BY make, model  
    ORDER BY make, cars_sold DESC;


    make	model	cars_sold
    Audi	A3	65
    Audi	A4	65
    Audi	Q3	55
    Audi	A6	46
    Audi	Q2	37
    BMW	320	19
    BMW	520	19
    BMW	530	15
    BMW	330	14
    BMW	X1	13
    Ford	Fiesta	223
    Ford	Focus	181
    Ford	Kuga	116
    Ford	Puma	112
    Ford	EcoSport	60
    Hyundai	i10	147
    Hyundai	i30	107
    Hyundai	i20	92
    Hyundai	Tucson	71
    Hyundai	Kona	53
    Mercedes-Benz	C 180	27
    Mercedes-Benz	Vito	21
    Mercedes-Benz	C 200	17
    Mercedes-Benz	C 220	17
    Mercedes-Benz	A 180	16
    Opel	Corsa	258
    Opel	Astra	60
    Opel	Mokka	58
    Opel	Crossland X	36
    Opel	Combo	23
    Renault	Clio	199
    Renault	Twingo	174
    Renault	Captur	58
    Renault	Megane	50
    Renault	ZOE	38
    SEAT	Leon	128
    SEAT	Ibiza	92
    SEAT	Ateca	57
    SEAT	Arona	57
    SEAT	Tarraco	21
    Skoda	Fabia	295
    Skoda	Octavia	133
    Skoda	Kamiq	79
    Skoda	Scala	60
    Skoda	Karoq	48
    Volkswagen	Golf	190
    Volkswagen	Tiguan	153
    Volkswagen	Polo	102
    Volkswagen	T-Roc	100
    Volkswagen	T-Cross	74

### Which OfferType Make The Most For Cars Sold ?


    SELECT  
        offer_type,  
        COUNT(*) AS total_cars_sold  
    FROM car_post_process  
    GROUP BY offer_type  
    ORDER BY total_cars_sold DESC;

![offertype](img/offertype.png)

### Which OfferType Make The Most For Cars Sold By Brand ? 


    SELECT  
        make,  
        offer_type,  
        COUNT(*) AS total_cars_sold  
    FROM car_post_process  
    WHERE make IN ('Volkswagen', 'Audi', 'Ford', 'Mercedes-Benz', 'Skoda', 'Renault', 'Hyundai', 'BMW', 'Opel', 'SEAT')  
    GROUP BY make, offer_type  
    ORDER BY make, total_cars_sold DESC;


    make	offer_type	total_cars_sold
    Audi	Used	289
    Audi	Demonstration	108
    Audi	Employee's car	74
    Audi	Pre-registered	26
    
    BMW	Used	131
    BMW	Employee's car	54
    BMW	Demonstration	25
    BMW	Pre-registered	1
    
    Ford	Used	309
    Ford	Employee's car	299
    Ford	Pre-registered	193
    Ford	Demonstration	112
    
    Hyundai	Pre-registered	315
    Hyundai	Demonstration	130
    Hyundai	Used	70
    Hyundai	Employee's car	12
    Hyundai	New	1
    
    Mercedes-Benz	Demonstration	151
    Mercedes-Benz	Employee's car	101
    Mercedes-Benz	Used	67
    Mercedes-Benz	Pre-registered	4
    
    Opel	Used	239
    Opel	Pre-registered	132
    Opel	Demonstration	71
    Opel	Employee's car	63
    
    Renault	Demonstration	257
    Renault	Used	243
    Renault	Pre-registered	147
    Renault	Employee's car	4
    
    SEAT	Pre-registered	147
    SEAT	Used	110
    SEAT	Demonstration	87
    SEAT	Employee's car	16
    SEAT	New	6
    
    Skoda	Pre-registered	306
    Skoda	Used	179
    Skoda	Demonstration	137
    Skoda	Employee's car	74
    Skoda	New	2
    
    Volkswagen	Used	510
    Volkswagen	Demonstration	219
    Volkswagen	Employee's car	201
    Volkswagen	Pre-registered	137

### Which Gear Make The Most For Cars Sold ?


    SELECT  
        gear,  
        COUNT(*) AS total_cars_sold  
    FROM car_post_process  
    GROUP BY gear  
    ORDER BY total_cars_sold DESC;


![geartype](img/geartype.png)

### Which Gear Make The Most For Cars Sold By Each Brand?


    SELECT  
        make,  
        gear,  
        COUNT(*) AS total_cars_sold  
    FROM car_post_process  
    WHERE make IN ('Volkswagen', 'Audi', 'Ford', 'Mercedes-Benz', 'Skoda', 'Renault', 'Hyundai', 'BMW', 'Opel', 'SEAT')  
    GROUP BY make, gear  
    ORDER BY make, total_cars_sold DESC;


    make	gear	total_cars_sold
    Audi	Automatic	443
    Audi	Manual	54
    
    BMW	Automatic	198
    BMW	Manual	13
    
    Ford	Manual	582
    Ford	Automatic	331
    
    Hyundai	Manual	313
    Hyundai	Automatic	215
    
    Mercedes-Benz	Automatic	295
    Mercedes-Benz	Manual	28
    
    Opel	Manual	334
    Opel	Automatic	171
    
    Renault	Manual	403
    Renault	Automatic	248
    
    SEAT	Manual	188
    SEAT	Automatic	178
    
    Skoda	Manual	426
    Skoda	Automatic	272
    
    Volkswagen	Automatic	651
    Volkswagen	Manual	416

## Visualization

    Used Power BI To Make The Visualizations Which Will Support Our Conclusions

![dash1](https://drive.google.com/file/d/1B7F63My02TjoLYBKkQ5VqayCgoxqvyE2/view)

    Base On Queries Result And Graphs Provided We Found That :
        The Leading Car Brand From 2018 To 2021 Was :
            Volkswagen  Making Up To 13.78% From The Market Revenu 
            and The Most Saled Model Was GolF

-    We Noticed That Even Ford Sold More Cars Than Audi But Audi Have A Bigger Share In Markt Revenue We Can
      Explain That By Considring Audi As Premium Car Brand Which Can Effect The Prices For Each Car
      We Can Apply The Same Principle For Skoda and Renault

-    We Noticed That Even BMW , Mercedes-Benz And Audi Are German Car Brands There Sales Fall Back By
     Comparise Them With  Other Foreign Car Brands Like Ford , Skoda ,Renault For Reasons I will Explain Later

![dash2](https://raw.githubusercontent.com/fawzi177/repo/main/img/dash2.png)

- We Noticed That The Types Of Gear For Cars Sold are Divided Semi-equal For Each Manual and
  Automatic With Very Little To None In Semi-automatic

- We Noticed That German Cars Brand Like Audi , BMW , Mercedes Benz Were Sold Automatic Cars More Than Manual
But The Most Sold Cars Brands Sold More Cars With Manual Than Automatic


- We Noticed in Offer Type Chart The The Shares are divided Semi-equal For Each Offer Except New Cars
  We Can Explain That With The Price - The Price For Used Car Always is lesser Than The New Car Even For The
  Same Model

### Studying The Most Sold Car Modle : Volkswagen Golf

by Showing The Technical Sheet For Golf Cars :
https://newspress-vwusamedia.s3.amazonaws.com/documents%2Foriginal%2F8689-2018VolkswagenGolfTechnicalSpecifications.pdf

1. Volkswagen’s Strong Presence in Germany
Volkswagen (VW) is a German automaker with deep roots in the country. The brand is associated with reliability, engineering excellence, and strong national loyalty. Many German buyers prefer Volkswagen models, including the Golf, due to trust in domestic brands.

2. Ideal for German Driving Conditions
The Volkswagen Golf is known for its fuel efficiency, making it a great choice for both city driving and the Autobahn.
It comes in multiple engine configurations (petrol, diesel, hybrid, and electric), offering options for different driving preferences.
It balances compact size for urban use with enough interior space for comfort.
3. Popularity in the Company Car Market
Many German companies provide employees with Volkswagen Golf models as company cars.
The Golf’s reputation for efficiency, safety, and resale value makes it a preferred choice for fleet purchases.
4. German Driving Culture and Practicality
German consumers favor hatchbacks over sedans, as they offer a mix of space, performance, and practicality.
The Golf GTI and Golf R versions provide sporty performance while still being practical for daily use.
5. Strong Resale Value & Longevity
The Golf has a high resale value in Germany, making it a solid long-term investment.
Volkswagen models, especially the Golf, are durable and have a strong aftermarket support network.


### Final Thoughts

Customers Are Always Looking For Cars Thats Give Them Best Values For There Money
By Providing Cars with Competitive Price & Features  while still maintaining high-quality engineering. Advanced safety features, technology upgrades, and driver assistance systems to  make them appealing to a wide range of Buyers.

### Author :


Fawzi Jabbour